In [1]:
from keras.applications import InceptionV3
from keras.applications import Xception # TensorFlow ONLY
from keras.applications import inception_resnet_v2
from keras.applications import VGG16
from keras.applications import VGG19
from keras.applications import imagenet_utils
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
import matplotlib.pyplot as plt
import collections
from PIL import Image
import operator
import numpy as np
import argparse
import cv2
from keras.layers.convolutional import AtrousConvolution1D
from keras.utils import to_categorical
from keras.layers import Input, Dropout, BatchNormalization, Activation, Add, Conv2DTranspose, ZeroPadding1D
import keras
from keras.layers import Conv1D, MaxPooling1D, Embedding, Merge, Dropout, UpSampling1D
from keras.models import Model, Sequential
from keras.layers import Input, concatenate, LSTM, Dense, RepeatVector, Embedding, TimeDistributed
from keras.layers import LSTM, GRU, MaxPooling1D
from keras.layers import RepeatVector
from keras.layers import Dense, Reshape
from keras.layers.core import Activation, Dense, Dropout, Flatten, Lambda
from tensorflow.python.keras.layers import Layer
import matplotlib.pyplot as plt
import re
import numpy as np
from keras.models import load_model
from keras.preprocessing.text import text_to_word_sequence
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D, GlobalAveragePooling2D
import keras.backend as K

Using TensorFlow backend.


In [76]:
import pandas as pd
import scipy

In [3]:
df_map = pd.read_csv('FeaturesMapping2.csv', header = None)

In [4]:
df = pd.read_csv('CategoryFeatures2.csv', header = None)

In [5]:
df[(df[0] == 11182)][3].unique()

array(['Назначение', 'Материал корпуса', 'Материал фасада', 'Тип шкафа',
       'Тип фасада шкафа', 'Тип мебели', 'Количество внешних полок, шт',
       'Количество внутренних полок, шт', 'Количество дверок, шт',
       'Количество выдвижных ящиков, шт', 'Цвет', 'Страна производитель',
       'Гарантийный срок, мес', 'Ширина, мм', 'Высота, мм', 'Длина, мм',
       'Глубина полки, мм', 'Высота полки, мм', 'Корпус', 'Фасады',
       'Полки', 'Декоративные элементы', 'Антресоль',
       'Тип направляющих для выдвижных ящиков',
       'Наличие декоративной подсветки',
       'Максимально допустимая нагрузка, кг', 'Вес, кг', 'Замок',
       'Цвет покрытия', 'Центральный замок', 'Вес нагрузки на полку',
       'Толщина каркаса',
       'Возможность закрытия дверей отдельно друг от друга',
       'Цвет дверей', 'Тип дверей', 'Материал дверей',
       'Трейзер (кассовое отделение с доп.ключом)', 'Длина шнура (м)',
       'Производительность воздуха', 'Материал изготовления',
       'Глубина ящ

In [6]:
categories = df[0].unique()
len_cat = len(categories)

In [7]:
ncategories = df[1].unique()
len(ncategories)

2674

In [8]:
df[(df[1] == 'Трансформаторы') & (df[3] == 'Страна производитель')][5].unique()

array(['Австралия', 'Австрия', 'Азад Кашмир', 'Азербайджан', 'Албания',
       'Алжир', 'Американское Самоа', 'Аомынь', 'Аргентина', 'Армения',
       'Афганистан', 'Багамские Острова', 'Бангладеш', 'Барбадос',
       'Бахрейн', 'Беларусь', 'Бельгия', 'Бенин', 'Болгария', 'Боливия',
       'Бразилия', 'Бруней', 'Буркина-Фасо', 'Вануату', 'Великобритания',
       'Венгрия', 'Восточный Тимор', 'Вьетнам', 'Гавайи', 'Гаити', 'Гана',
       'Гватемала', 'Германия', 'Гондурас', 'Гонконг', 'Гренада',
       'Греция', 'Грузия', 'Дания', 'Демократическая Республика Конго',
       'Доминиканская Республика', 'Египет', 'Замбия', 'Зимбабве',
       'Израиль', 'Индия', 'Индонезия', 'Иордания', 'Иран', 'Ирландия',
       'Испания', 'Италия', 'Кабо-Верде', 'Казахстан', 'Камбоджа',
       'Камерун', 'Канада', 'Катар', 'Кения', 'Кипр', 'Киргизия', 'Китай',
       'Кокосовые острова', 'Колумбия', 'Коморские Острова', 'Коста-Рика',
       'Кот-д’Ивуар', 'Куба', 'Кувейт', 'Латвия', 'Лесото', 'Ливан',
    

In [9]:
a = set()
for x,y in zip(df[0],df[1]):
    a.add(str(y)+'_'+str(x))

In [10]:
mx = 0
for cat in categories:
    m = len(df[df[0] == cat][2].unique())
    if m > mx:
        mx = m

In [11]:
mx # макс колл наз хар-к

93

In [12]:
m=0
for cat in categories:
    names = df[df[0] == cat][2].unique()
    for name in names:
        m = len(df[(df[0] == cat)&(df[2] == name)])
        if m > mx:
            mx = m

In [13]:
mx

1413

In [12]:
len_n = 93
len_v = 1413
global_dict = {}

In [13]:
for ic, cat in enumerate(categories):
    global_dict[cat] = [[],{}]
    global_dict[cat][0] = to_categorical(ic, num_classes=len_cat)
    
    names = df[df[0] == cat][2].unique()
    for i_n, name in enumerate(names):
        global_dict[cat][1][name] = [[],{}]
        global_dict[cat][1][name][0] = to_categorical(i_n, num_classes=len_n)
        
        values = df[(df[0] == cat)&(df[2] == name)][4].unique()
        for iv, value in enumerate(values):
            global_dict[cat][1][name][1][value] = to_categorical(iv, num_classes=len_v)
            

In [14]:
global_dict[77][1][17991][1][378733].shape

(1413,)

In [15]:
df_map[30000:30100]

,0,1,2,3,4,5,6,7,8
30000,24041,"Ширина, мм",570351,575,Ширина (Ш),575 мм,11182,Шкафы,"{Шкафы,Шкафы, комоды, полки,Мебель,Дом и дача}"
30001,24041,"Ширина, мм",570432,600,Ширина,600 мм,11182,Шкафы,"{Шкафы,Шкафы, комоды, полки,Мебель,Дом и дача}"
30002,24041,"Ширина, мм",570432,600,Ширина (1),600 мм,11182,Шкафы,"{Шкафы,Шкафы, комоды, полки,Мебель,Дом и дача}"
30003,24041,"Ширина, мм",570432,600,Ширина (Габарит X),600 мм,11182,Шкафы,"{Шкафы,Шкафы, комоды, полки,Мебель,Дом и дача}"
30004,24041,"Ширина, мм",570432,600,Ширина (Ш),600 мм,11182,Шкафы,"{Шкафы,Шкафы, комоды, полки,Мебель,Дом и дача}"
30005,24041,"Ширина, мм",570434,320,Ширина,320 мм,11182,Шкафы,"{Шкафы,Шкафы, комоды, полки,Мебель,Дом и дача}"
30006,24041,"Ширина, мм",570434,320,Ширина (1),320 мм,11182,Шкафы,"{Шкафы,Шкафы, комоды, полки,Мебель,Дом и дача}"
30007,24041,"Ширина, мм",570434,320,Ширина (Габарит X),320 мм,11182,Шкафы,"{Шкафы,Шкафы, комоды, полки,Мебель,Дом и дача}"
30008,24041,"Ширина, мм",570434,320,Ширина (Ш),320 мм,11182,Шкафы,"{Шкафы,Шкафы, комоды, полки,Мебель,Дом и дача}"
30009,24041,"Ширина, мм",570455,800,Ширина,800 мм,11182,Шкафы,"{Шкафы,Шкафы, комоды, полки,Мебель,Дом и дача}"


In [18]:
df_map[6]

0         12051
1         12051
2         12051
3         12051
4         12051
5         12051
6         12051
7         12051
8         12051
9         12051
10        12051
11        12051
12        12051
13        12051
14        12051
15        12051
16        12051
17        12051
18        12051
19        12051
20        12051
21        12051
22        12051
23        12051
24        12051
25        12052
26        12052
27        12052
28        12052
29        12052
          ...  
914086    12485
914087    12485
914088    12485
914089    12485
914090    12485
914091    12485
914092    12485
914093    12485
914094    12485
914095    12485
914096    12485
914097    12485
914098    12485
914099    12485
914100    12485
914101    12485
914102    12485
914103    12485
914104    12485
914105    12485
914106    12485
914107     6321
914108     6321
914109     6321
914110    36427
914111     2432
914112     2432
914113     2432
914114     2432
914115     2432
Name: 6, Length: 914116,

In [19]:
global_dict[12051]

[array([0., 0., 0., ..., 0., 0., 0.]),
 {2581: [array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0.]),
   {25044: array([1., 0., 0., ..., 0., 0., 0.]),
    25045: array([0., 1., 0., ..., 0., 0., 0.]),
    25046: array([0., 0., 1., ..., 0., 0., 0.]),
    25047: array([0., 0., 0., ..., 0., 0., 0.]),
    25048: array([0., 0., 0., ..., 0., 0., 0.]),
    25049: array([0., 0., 0., ..., 0., 0., 0.]),
    25050: array([0., 0., 0., ..., 0., 0., 0.]),
    25051: array([0., 0., 0., ..., 0., 0., 0.])}],
  2582: [array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0

In [20]:
# for i,val in df_map.iterrows():
#     print(val[6], val[2])

## --------------------------------------------------------------------------------------------

In [16]:
chars = ' абвгдеёжзийклмнопрстуфхцчшщъыьэюяabcdefghijklmnopqrstuvwxyz0123456789-,;.’!?:\'"/\\|_@#$%°«^&ø*~№`+—-=<>()[]{}'

In [17]:
char_to_ix = {char:ix for ix, char in enumerate(chars)}
VOCAB_SIZE = len(chars)
VOCAB_SIZE

109

In [18]:
rus_chars=['а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п','р','с', 'т','у', 'ф','х', 'ц','ч','ш', 'щ','ъ','ы','ь','э','ю','я', 'ё',]
len(rus_chars)

33

In [19]:
eng_chars = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z',]
len(eng_chars)

26

In [20]:
digits = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',]

In [21]:
def get_mean_vector(ofr):
    means = np.zeros((len(ofr)))
    ofr = ofr.split()
    itr = 0
    for of in ofr:
        sum_ = 0
        for i,t in enumerate(of):
            sum_+=char_to_ix[t]
            
        sum_/=len(of)
        means[itr:itr+len(of)] = sum_ * len(chars)
        itr = itr+len(of)+1
    return means
    
def get_b_e_vector(ofr):
    means = np.zeros((len(ofr), 4))
    for i, sym in enumerate(ofr):
        if sym == ' ':
            means[i] = to_categorical(1, num_classes=4) 
        if sym in rus_chars:
            means[i] = to_categorical(2, num_classes=4) 
        if sym in eng_chars:
            means[i] = to_categorical(3, num_classes=4) 
    
    means1 = np.zeros((len(ofr), 4))
    ofr = ofr.split()
    itr = 0
    for of in ofr:
        means1[itr] = to_categorical(1, num_classes=4)
        means1[itr+len(of)-1] = to_categorical(2, num_classes=4)       
        itr = itr+len(of)+1
        if (itr-1) < len(means1):
            means1[itr-1]=to_categorical(-1, num_classes=4)   
    
    
    return np.concatenate((means,means1),axis=-1)


In [22]:
def get_X(ofr):
    x_test = np.zeros((1, len(ofr)))#,len(chars)+4
    x_test_1 = np.zeros((1, len(ofr),len(chars)+8))#
    mean_vector = get_b_e_vector(ofr)
    for i,t in enumerate(ofr):
        if t not in char_to_ix:
            t = ' '
        one_hot = to_categorical(char_to_ix[t], num_classes=len(chars))
        m = mean_vector[i]
        one_hot = np.append(one_hot,[m])
        x_test_1[0][i] = one_hot
        x_test[0][i] = char_to_ix[t]#one_hot
        
    return x_test_1, x_test


In [23]:
batch_size = 32
length = 914116
clength = int(length/batch_size)+1

def gen_X():
    while 1:
        for i,itms in df_map.iterrows():
            cat = itms[6]
            val = itms[2]
            name = itms[0]

            vec_cat = global_dict[cat][0]
            vec_name = global_dict[cat][1][name][0]
            vec_value = global_dict[cat][1][name][1][val]

            ncat = itms[7]
            ncat = ncat.lower().replace('}','').replace('{','')
            nname = itms[4]
            nname = nname.lower()
            nvalue = str(itms[5])
            nvalue = nvalue.lower()

            x_str = ncat+'_'+nname+'_'+nvalue
            x1,x = get_X(x_str)

            yield [x,x1],[ np.array([vec_cat]), np.array([vec_name]), np.array([vec_value]) ]
    
    

In [24]:
from sklearn import utils
utils.shuffle(df_map)

,0,1,2,3,4,5,6,7,8
365133,38085,Размеры игрушки,736213,15 х 4.5 х 15 см.,Размер автовоза,15 х 4.5 х 15 см.,2254,Игровые наборы,"{Игровые наборы,Детские игрушки и игры,Детские..."
539671,38085,Размеры игрушки,736546,5 х 6.5 см.,Размер корзинки,5 х 6.5 см.,2254,Игровые наборы,"{Игровые наборы,Детские игрушки и игры,Детские..."
556478,38085,Размеры игрушки,736578,61 х 32 х 65-96 см.,Размер барельефа,61 х 32 х 65-96 см.,2254,Игровые наборы,"{Игровые наборы,Детские игрушки и игры,Детские..."
874958,39267,Вес,748994,168 г.,Общий вес пластилина,168 г.,2743,Лепка и литье,"{Лепка и литье,Хобби и творчество,Обучение и т..."
49611,26446,Возрастная группа,515075,От 3 лет,Возраст,от 3 до 5 лет,2255,Куклы и пупсы,"{Куклы и пупсы,Игрушки для девочек,Детские игр..."
509271,38085,Размеры игрушки,736487,60 см.,Размер 1 игрушки,60 см.,2254,Игровые наборы,"{Игровые наборы,Детские игрушки и игры,Детские..."
248125,38085,Размеры игрушки,735994,0.5 x 42 x 30 см.,Размер каждого трафарета,0.5 x 42 x 30 см.,2254,Игровые наборы,"{Игровые наборы,Детские игрушки и игры,Детские..."
765198,38085,Размеры игрушки,737003,10 см х 8 см х 2.5 см.,Размер щенка,10 см х 8 см х 2.5 см.,2254,Игровые наборы,"{Игровые наборы,Детские игрушки и игры,Детские..."
690038,38085,Размеры игрушки,736835,43 х 48 клеток.,Размер барельефа,43 х 48 клеток.,2254,Игровые наборы,"{Игровые наборы,Детские игрушки и игры,Детские..."
464980,38085,Размеры игрушки,736403,32 х 23 см.,Размер лягушонка,32 х 23 см.,2254,Игровые наборы,"{Игровые наборы,Детские игрушки и игры,Детские..."


In [25]:
def create_conv_model(start_neurons, cnt_classes=3):
    input_sparse = Input((None, len(chars)+8)) # Input((None, len(chars)+4))
    
    input_size = None
    inputs = Input(shape=(input_size,), name='sent_input', dtype='int64')
    x = Embedding(len(chars)+8, 256, input_length=input_size)(inputs)
    
    x = keras.layers.concatenate([x, input_sparse])
    
    frst_conv = Conv1D(32, 1, activation='relu', padding='same', dilation_rate=1)(x)
    
    conv1 = Conv1D(64, 2, activation='relu', padding='same', dilation_rate=1)(x)
    conv1 = BatchNormalization()(conv1)
    conv1 = MaxPooling1D(3, strides=1, padding='same')(conv1)   
    conv1 = Conv1D(64, 2, activation='relu', padding='same', dilation_rate=2)(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = MaxPooling1D(3, strides=1, padding='same')(conv1)   
    conv1 = Conv1D(128, 2, activation='relu', padding='same', dilation_rate=3)(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = MaxPooling1D(3, strides=1, padding='same')(conv1)   
    conv1 = Conv1D(128, 2, activation='relu', padding='same', dilation_rate=4)(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = MaxPooling1D(3, strides=1, padding='same')(conv1)   
    conv1 = Conv1D(256, 2, activation='relu', padding='same', dilation_rate=5)(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = MaxPooling1D(3, strides=1, padding='same')(conv1)   
    conv1 = Conv1D(512, 2, activation='relu', padding='same', dilation_rate=6)(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = MaxPooling1D(3, strides=1, padding='same')(conv1) 
    conv1 = Conv1D(512, 2, activation='relu', padding='same', dilation_rate=6)(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = MaxPooling1D(3, strides=1, padding='same')(conv1) 
    conv1 = Conv1D(1024, 2, activation='relu', padding='same', dilation_rate=7)(conv1)
    end = BatchNormalization()(conv1)
    
    avg = GlobalAveragePooling1D()(end)

    
    conv_cat = Dense(len_cat, activation='softmax')(avg)
    conv_name = Dense(len_n, activation='softmax')(avg)
    conv_val = Dense(len_v, activation='softmax')(avg)
    
    model = Model(inputs=[inputs, input_sparse], outputs = [conv_cat, conv_name, conv_val])
    
    return model
    
model = create_conv_model(256)

In [26]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sent_input (InputLayer)         (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 256)    29952       sent_input[0][0]                 
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, None, 117)    0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, None, 373)    0           embedding_1[0][0]                
                                                                 input_1[0][0]                    
__________

In [31]:
model.compile(loss='binary_crossentropy', optimizer='adam')

In [90]:
model.fit_generator(gen_X(), length, epochs = 2)

Epoch 1/2
914116/914116 [==============================] - 30215s 33ms/step - loss: 0.0623 - dense_1_loss: 0.0013 - dense_2_loss: 0.0444 - dense_3_loss: 0.0166
Epoch 2/2
695140/914116 [=====================>........] - ETA: 2:00:01 - loss: 0.1113 - dense_1_loss: 0.0025 - dense_2_loss: 0.0861 - dense_3_loss: 0.0226- ETA: 2:00:06 - loss: 0.1113 - dense_1_loss: 0.0025 - dense_2_loss: 0.0861 - dense_3_loss: 0. - ETA: 2:00:05 - loss: 0.1113 - dense_1_loss: 0.0025 - dense_2_loss: 0.0861 - dense_3 - ETA: 2:00:04 - loss: 0.1113 - dense_1_loss: 0.002

KeyboardInterrupt: 

In [40]:
model.save('ver_1.hdf5')

In [27]:
from keras.models import load_model
model = load_model('ver_1.hdf5')

In [28]:
model_vec = Model(inputs=model.input, outputs = model.layers[-4].output)

In [86]:
def get_vec(ofr) :
    x_test = np.zeros((1, len(ofr)))#,len(chars)+4
    x_test_1 = np.zeros((1, len(ofr),len(chars)+8))#
    mean_vector = get_b_e_vector(ofr)
    for i,t in enumerate(ofr):
        if t not in char_to_ix:
            t=' '
        one_hot = to_categorical(char_to_ix[t], num_classes=len(chars))
        m = mean_vector[i]
        one_hot = np.append(one_hot,[m])
        x_test_1[0][i] = one_hot
        x_test[0][i] = char_to_ix[t]

    # pred = model.predict([x_test,x_test_1])[0]
    return model_vec.predict([x_test,x_test_1])

In [109]:
pr = get_vec('Шкафы_Длина_570'.lower())

In [110]:
pr.shape

(1, 1024)

In [111]:
charcter = ''
m = 1.0
for i, chrs in df[df[1]=='Шкафы'].iterrows():
    chr_v = chrs[1]+'_'+chrs[3]+'_'+chrs[5]
    chr_v = chr_v.lower()
    res = scipy.spatial.distance.cosine(pr, get_vec(chr_v))
    if res < m:
        m = res
        charcter = chr_v

In [112]:
charcter

'шкафы_корпус_бамбук'

In [ ]:
pr1 = get_vec('Шкафы,Шкафы, комоды, полки,Мебель,Дом и дача_Шкафы_Ширина_800'.lower())
pr2 = get_vec('Шкафы,Шкафы, комоды, полки,Мебель,Дом и дача_Шкафы_Ширина_габариты(х)_600 мм'.lower())

In [94]:
df[(df[1]=='Шкафы')& (df[3]=='Глубина ящика')]

,0,1,2,3,4,5,6
103230,11182,Шкафы,47142,Глубина ящика,802717,384,"{Шкафы,Шкафы, комоды, полки,Мебель,Дом и дача}"


In [104]:
df[(df[1]=='Смесители')]

,0,1,2,3,4,5,6
141901,11680,Смесители,18041,Страна производитель,379275,Австралия,"{Смесители,Сантехника и водоснабжение,Строител..."
141902,11680,Смесители,18041,Страна производитель,379276,Австрия,"{Смесители,Сантехника и водоснабжение,Строител..."
141903,11680,Смесители,18041,Страна производитель,379277,Беларусь,"{Смесители,Сантехника и водоснабжение,Строител..."
141904,11680,Смесители,18041,Страна производитель,379278,Бельгия,"{Смесители,Сантехника и водоснабжение,Строител..."
141905,11680,Смесители,18041,Страна производитель,379279,Болгария,"{Смесители,Сантехника и водоснабжение,Строител..."
141906,11680,Смесители,18041,Страна производитель,379280,Венгрия,"{Смесители,Сантехника и водоснабжение,Строител..."
141907,11680,Смесители,18041,Страна производитель,379281,Германия,"{Смесители,Сантехника и водоснабжение,Строител..."
141908,11680,Смесители,18041,Страна производитель,379282,Гонконг,"{Смесители,Сантехника и водоснабжение,Строител..."
141909,11680,Смесители,18041,Страна производитель,379284,Индия,"{Смесители,Сантехника и водоснабжение,Строител..."
141910,11680,Смесители,18041,Страна производитель,379285,Испания,"{Смесители,Сантехника и водоснабжение,Строител..."


In [94]:
id_cat = 0
for k,v in global_dict.items():
    if pr[0].argmax() == v[0].argmax():
        print(k)
        id_cat = k
        break

id_n = 0
for k,v in global_dict[id_cat][1].items():
    if pr[1].argmax() == v[0].argmax():
        print(k)
        id_n = k
        break


2748
27045


In [95]:
df[(df[0] == id_cat)&(df[2] == id_n)]

,0,1,2,3,4,5,6
51940,2748,Рисование и гравюры,27045,Длина (см),534144,40,"{Рисование и гравюры,Хобби и творчество,Обучен..."
51941,2748,Рисование и гравюры,27045,Длина (см),534145,3,"{Рисование и гравюры,Хобби и творчество,Обучен..."
51942,2748,Рисование и гравюры,27045,Длина (см),534146,81,"{Рисование и гравюры,Хобби и творчество,Обучен..."
51943,2748,Рисование и гравюры,27045,Длина (см),534491,23,"{Рисование и гравюры,Хобби и творчество,Обучен..."
51944,2748,Рисование и гравюры,27045,Длина (см),534702,25,"{Рисование и гравюры,Хобби и творчество,Обучен..."
51945,2748,Рисование и гравюры,27045,Длина (см),534706,22,"{Рисование и гравюры,Хобби и творчество,Обучен..."
51946,2748,Рисование и гравюры,27045,Длина (см),534710,16,"{Рисование и гравюры,Хобби и творчество,Обучен..."
51947,2748,Рисование и гравюры,27045,Длина (см),534714,20,"{Рисование и гравюры,Хобби и творчество,Обучен..."
51948,2748,Рисование и гравюры,27045,Длина (см),535917,50,"{Рисование и гравюры,Хобби и творчество,Обучен..."
51949,2748,Рисование и гравюры,27045,Длина (см),536086,61,"{Рисование и гравюры,Хобби и творчество,Обучен..."


In [77]:
utils.shuffle(df_map)

,0,1,2,3,4,5,6,7,8
440900,38085,Размеры игрушки,736356,22 x 24 см.,Размер 1 дощечки,22 x 24 см.,2254,Игровые наборы,"{Игровые наборы,Детские игрушки и игры,Детские..."
672343,38085,Размеры игрушки,736799,10.5 x 4 x 4 см.,Размер собранного гаража,10.5 x 4 x 4 см.,2254,Игровые наборы,"{Игровые наборы,Детские игрушки и игры,Детские..."
324518,38085,Размеры игрушки,736136,14.5 x 22 x 8 см.,Размер лесопилки,14.5 x 22 x 8 см.,2254,Игровые наборы,"{Игровые наборы,Детские игрушки и игры,Детские..."
48122,26302,Материал,748672,"резина, текстиль",Из чего сделана игрушка (состав),"резина, текстиль.",2265,Игрушки для ванной,"{Игрушки для ванной,Игрушки для малышей,Детски..."
681055,38085,Размеры игрушки,736818,20 х 11 x 2 см.,Размер буквы,20 х 11 x 2 см.,2254,Игровые наборы,"{Игровые наборы,Детские игрушки и игры,Детские..."
660002,38085,Размеры игрушки,736774,18 x 6 x 10 см.,Размер кэтмобиля,18 x 6 x 10 см.,2254,Игровые наборы,"{Игровые наборы,Детские игрушки и игры,Детские..."
717934,38085,Размеры игрушки,736899,4 х 4 х 4.5 см.,Размер совка,4 х 4 х 4.5 см.,2254,Игровые наборы,"{Игровые наборы,Детские игрушки и игры,Детские..."
637215,38085,Размеры игрушки,736729,37 х 17 х 19 см.,Размер ласт,37 х 17 х 19 см.,2254,Игровые наборы,"{Игровые наборы,Детские игрушки и игры,Детские..."
115788,33248,Количество страниц,687308,224,Количество страниц,224,14473,Литература Средних веков и Возрождения,"{Литература Средних веков и Возрождения,Зарубе..."
19286,18211,"Объем, л",624363,141,"Объем, л",141,11681,Ванны,"{Ванны,Сантехника и водоснабжение,Строительств..."
